In [9]:
!pip -q install langchain==0.2.1 openai
!pip -q install duckduckgo-search

Setting up some keys

In [10]:
!pip show langchain

Name: langchain
Version: 0.2.1
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [11]:
pip show openai

Name: openai
Version: 1.30.5
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: 



# Custom Tools & Agents 🤖

In [14]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.2 MB/s eta 0:00:00


In [ ]:
#@title Keys

import os

os.environ["OPENAI_API_KEY"] = "your openai api key"

In [16]:
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [45]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

## Standard Tool

In [19]:
%pip install --upgrade --quiet  duckduckgo-search langchain-community

In [20]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import Tool
from langchain.tools import BaseTool

search = DuckDuckGoSearchRun()

# defining a single tool
search_tool = Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    )


In [21]:
# # Import the necessary modules
# from langchain import OpenAI
# from langchain.chat_models import ChatOpenAI
# from langchain.chains.conversation.memory import ConversationBufferWindowMemory
# from langchain_community.tools import DuckDuckGoSearchRun
# from langchain.agents import Tool
# from langchain.tools import BaseTool

# # Set up the turbo LLM
# turbo_llm = ChatOpenAI(
#     temperature=0,
#     model_name='gpt-3.5-turbo'
# )

# # Initialize the DuckDuckGo search tool
# search = DuckDuckGoSearchRun()

# # Define a list of tools

# search_tool = Tool(
#         name="search",
#         func=search.run,
#         description="useful for when you need to answer questions about current events. You should ask targeted questions"
#     )


## Custom tools

In [22]:
def meaning_of_life(input=""):
    return 'The meaning of life is 42 if rounded but is actually 42.17658'

In [23]:

life_tool = Tool(
    name='Meaning of Life',
    func= meaning_of_life,
    description="Useful for when you need to answer questions about the meaning of life. input should be MOL "
)

## Random number

In [24]:
import random

def random_num(input=""):
    return random.randint(0,5)

In [25]:
random_num()

4

In [26]:
random_tool = Tool(
    name='Random number',
    func= random_num,
    description="Useful for when you need to get a random number. input should be 'random'"
)

## Creating an agent

# ConversationBufferWindowMemory

The `ConversationBufferWindowMemory` is a type of memory used in conversational AI agents, specifically in the Langchain library. It's a crucial component that enables the agent to keep track of the conversation history and respond accordingly.

Here's a breakdown of what it does:

**Buffer**: The buffer refers to a temporary storage area that holds a limited amount of data. In the context of conversational AI, the buffer stores the conversation history, including the user's input and the agent's responses.

**Window**: The window refers to a specific portion of the buffer that is currently being considered. Think of it as a sliding window that moves along the buffer as the conversation progresses. The window size determines how many previous turns of the conversation are taken into account when generating a response.

**Memory**: The memory component is responsible for storing and retrieving the conversation history. It's the brain of the conversational AI, allowing the agent to recall previous conversations and adapt its responses accordingly.

When you create a `ConversationBufferWindowMemory` instance, you can specify the following parameters:

* `memory_key`: a unique identifier for the memory instance
* `k`: the window size, which determines how many previous turns are considered
* `return_messages`: a boolean indicating whether to return the entire conversation history or just the latest message

By using a `ConversationBufferWindowMemory`, the conversational AI agent can:

1. Keep track of the conversation history
2. Respond contextually, taking into account previous turns of the conversation
3. Adapt its responses based on the user's input and previous interactions

This type of memory is particularly useful in chatbots, virtual assistants, and other conversational AI applications where context and history are essential for generating relevant and engaging responses.

In [27]:
from langchain.agents import initialize_agent

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)


custom_tools = [search_tool, random_tool, life_tool]

# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools = custom_tools,
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory

)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


**The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.**

# create_react_agent

The `create_react_agent` function is a part of the Langchain library, which is used to create a conversational AI agent that can respond to user input in a reactive manner.

Here's a breakdown of what the `create_react_agent` function does:

**Reactive Agent**: A reactive agent is a type of conversational AI that responds to user input in real-time, without requiring a specific goal or task to be accomplished. The agent's primary objective is to engage in conversation, provide information, and respond to user queries.

**create_react_agent**: This function creates an instance of a reactive agent, which can be used to engage in conversation with users. The function takes in several parameters, including:

* `agent_type`: The type of reactive agent to create (e.g., `chat-react-description`)
* `llm`: The language model to use for generating responses (e.g., `turbo_llm`)
* `tools`: A list of tools or plugins to use with the agent (e.g., `search_tool`, `random_tool`)
* `memory`: The memory component to use with the agent (e.g., `ConversationBufferWindowMemory`)
* `verbose`: A boolean indicating whether to print verbose output
* `max_iterations`: The maximum number of iterations to perform when generating a response
* `early_stopping_method`: The method to use for early stopping (e.g., `generate`)

Here's an example of how to use the `create_react_agent` function:
```python
from langchain.agents import create_react_agent

# Create a reactive agent
agent = create_react_agent(
    agent_type='chat-react-description',
    llm=turbo_llm,
    tools=[search_tool, random_tool],
    memory=ConversationBufferWindowMemory(memory_key='chat_history', k=3, return_messages=True),
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate'
)

# Use the agent to respond to user input
user_input = "What's the weather like today?"
response = agent(user_input)
print(response)
```
In this example, the `create_react_agent` function is used to create a reactive agent that can respond to user input. The agent is configured to use a specific language model, tools, and memory component. The `agent` instance can then be used to respond to user input, generating a response based on the conversation history and context.

In [48]:
conversational_agent("What time is it in London?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "I recommend using a world clock website or tool to check the current time in London."
}
```

> Finished chain.


{'input': 'What time is it in London?',
 'chat_history': [HumanMessage(content='What are the titles of the top stories on www.cbsnews.com/?'),
  AIMessage(content="The top stories on CBS News include: 1. Watch Live: Dr. Anthony Fauci testifies before Republican-led House panel 2. Who is Claudia Sheinbaum, Mexico's first woman to be elected president? 3. Hawaii volcano erupts, prompting officials to issue warning 4. Jury selection in Hunter Biden's federal gun trial begins 5. Biden executive order restricting asylum along border expected Tuesday"),
  HumanMessage(content='What time is it in London?'),
  AIMessage(content="I'm sorry, I don't have real-time capabilities to provide the current time in London. You can check the time using a clock, a world clock website, or a time zone converter tool."),
  HumanMessage(content='What time is it in London?'),
  AIMessage(content='I recommend using a world clock website or tool to check the current time in London.')],
 'output': 'I recommend us

In [29]:

conversational_agent("Can you give me a random number?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Random number",
    "action_input": "random"
}
```
Observation: 3
Thought:```json
{
    "action": "Final Answer",
    "action_input": "3"
}
```

> Finished chain.


{'input': 'Can you give me a random number?',
 'chat_history': [HumanMessage(content='What time is it in London?'),
  AIMessage(content='The current local time in London is 59 minutes ahead of apparent solar time. London is the capital of the United Kingdom. Latitude: 51.51. Longitude: -0.13; Population: 8,962,000. London time is 19:19 UTC on Friday, May 31, 2024. London is 6 hours ahead of Chicago, Illinois.')],
 'output': '3'}

In [30]:
conversational_agent("What is the meaning of life?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Meaning of Life",
    "action_input": "MOL"
}
```
Observation: The meaning of life is 42 if rounded but is actually 42.17658
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The meaning of life is 42 if rounded but is actually 42.17658"
}
```

> Finished chain.


{'input': 'What is the meaning of life?',
 'chat_history': [HumanMessage(content='What time is it in London?'),
  AIMessage(content='The current local time in London is 59 minutes ahead of apparent solar time. London is the capital of the United Kingdom. Latitude: 51.51. Longitude: -0.13; Population: 8,962,000. London time is 19:19 UTC on Friday, May 31, 2024. London is 6 hours ahead of Chicago, Illinois.'),
  HumanMessage(content='Can you give me a random number?'),
  AIMessage(content='3')],
 'output': 'The meaning of life is 42 if rounded but is actually 42.17658'}

In [49]:
# system prompt
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template

"Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.\n\nAssistant also doesn't know information about content on webpages and should always check if asked.\n\nOverall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assist

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

In [51]:
fixed_prompt = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.'''

In [50]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

In [52]:
conversational_agent("What is the meaning of life?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Meaning of Life",
    "action_input": "MOL"
}
```
Observation: The meaning of life is 42 if rounded but is actually 42.17658
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The meaning of life is 42 if rounded but is actually 42.17658"
}
```

> Finished chain.


{'input': 'What is the meaning of life?',
 'chat_history': [HumanMessage(content='What time is it in London?'),
  AIMessage(content="I'm sorry, I don't have real-time capabilities to provide the current time in London. You can check the time using a clock, a world clock website, or a time zone converter tool."),
  HumanMessage(content='What time is it in London?'),
  AIMessage(content='I recommend using a world clock website or tool to check the current time in London.'),
  HumanMessage(content='What time is it in London?'),
  AIMessage(content='I recommend using a world clock website or tool to check the current time in London.')],
 'output': 'The meaning of life is 42 if rounded but is actually 42.17658'}

## Using the Tool Class - lets make a useful tool


In [53]:
from bs4 import BeautifulSoup
import requests

def stripped_webpage(webpage):
    response = requests.get(webpage)
    html_content = response.text

    def strip_html_tags(html_content):
        soup = BeautifulSoup(html_content, "html.parser")
        stripped_text = soup.get_text()
        return stripped_text

    stripped_content = strip_html_tags(html_content)

    if len(stripped_content) > 4000:
        stripped_content = stripped_content[:4000]
    return stripped_content


In [54]:
stripped_webpage('https://www.google.com')


'GoogleSearch Images Maps Play YouTube News Gmail Drive More »Web History | Settings | Sign in\xa0Advanced searchAdvertisingBusiness SolutionsAbout Google© 2024 - Privacy - Terms   '

In [37]:
# webpage_tool = Tool(
#     name='Get Webpage',
#     func= stripped_webpage,
#     description="Useful for when you need to get the http from a specific webpage"
# )


In [55]:
class WebPageTool(BaseTool):
    name = "Get Webpage"
    description = "Useful for when you need to get the content from a specific webpage"

    def _run(self, webpage: str):
        response = requests.get(webpage)
        html_content = response.text

        def strip_html_tags(html_content):
            soup = BeautifulSoup(html_content, "html.parser")
            stripped_text = soup.get_text()
            return stripped_text

        stripped_content = strip_html_tags(html_content)
        if len(stripped_content) > 4000:
            stripped_content = stripped_content[:4000]
        return stripped_content

    def _arun(self, webpage: str):
        raise NotImplementedError("This tool does not support async")

page_getter = WebPageTool()

## Init the Agent again

In [56]:
fixed_prompt = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.

Assistant also doesn't know information about content on webpages and should always check if asked.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.'''

In [59]:
from langchain.prompts.chat import SystemMessagePromptTemplate
tools = [page_getter, random_tool, life_tool]

conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    memory=memory
)

In [60]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

In [61]:
conversational_agent.agent.llm_chain.prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.\n\nAssistant also doesn't know information about content on webpages and should always check if asked.\n\nOverall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specifi

In [62]:
conversational_agent.invoke("Is there an article about Clubhouse on https://techcrunch.com/? today")



> Entering new AgentExecutor chain...
```json
{
    "action": "Get Webpage",
    "action_input": "https://techcrunch.com/"
}
```
Observation: 























TechCrunch | Startup and Technology News



















































 



 


























Login





SearchStartupsVentureSecurityAICryptoAppsEventsStartup BattlefieldMoreFintechHardwareTransportationMedia & EntertainmentNewslettersPodcastsPartner ContentCrunchboard JobsContact UsSearch















Featured Article
Inside Apple’s efforts to build a better recycling robot


Brian Heater











 
		Media & Entertainment	
Spotify to increase premium pricing in the US to $11.99 per month


Paul Sawers





Featured Article
Unicorn-rich VC Wesley Chan owes his success to a Craigslist job washing lab beakers


Christine Hall





 
		Social	
French startup ten ten reinvents the walkie-talkie


Anna Heim





 
		Social	
X tweaks rules to formally allow adult content


Ivan Mehta








{'input': 'Is there an article about Clubhouse on https://techcrunch.com/? today',
 'chat_history': [HumanMessage(content='What time is it in London?'),
  AIMessage(content='I recommend using a world clock website or tool to check the current time in London.'),
  HumanMessage(content='What time is it in London?'),
  AIMessage(content='I recommend using a world clock website or tool to check the current time in London.'),
  HumanMessage(content='What is the meaning of life?'),
  AIMessage(content='The meaning of life is 42 if rounded but is actually 42.17658')],
 'output': 'There is no specific article about Clubhouse on TechCrunch today.'}

In [64]:
conversational_agent.invoke("What are the titles of the top stories on www.cbsnews.com/?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Get Webpage",
    "action_input": "https://www.cbsnews.com/"
}
```
Observation: 
CBS News - Breaking news, 24/7 live streaming news & top stories
















Latest

U.S.


World


Politics


Entertainment


HealthWatch


MoneyWatch


Investigations


Crime


Space


Sports



Local News

Baltimore


Bay Area


Boston


Chicago


Colorado


Detroit


Los Angeles


Miami


Minnesota


New York


Philadelphia


Pittsburgh


Sacramento


Texas



Live

CBS News 24/7


Baltimore


Bay Area


Boston


Chicago


Colorado


Detroit


Los Angeles


Miami


Minnesota


New York


Philadelphia


Pittsburgh


Sacramento


Texas



Shows

48 Hours


60 Minutes


America Decides


CBS Evening News


CBS Mornings


CBS News Eye on America


CBS News Mornings


CBS Reports


CBS Saturday Morning


The Daily Report


The Dish


Face the Nation


Here Comes the Sun


Person to Person


Sunday Morning


The Takeout


The Uplift


Weeke

{'input': 'What are the titles of the top stories on www.cbsnews.com/?',
 'chat_history': [HumanMessage(content='What is the meaning of life?'),
  AIMessage(content='The meaning of life is 42 if rounded but is actually 42.17658'),
  HumanMessage(content='Is there an article about Clubhouse on https://techcrunch.com/? today'),
  AIMessage(content='There is no specific article about Clubhouse on TechCrunch today.'),
  HumanMessage(content='What are the titles of the top stories on www.cbsnews.com/?'),
  AIMessage(content="The top stories on CBS News include: 1. Dr. Anthony Fauci testifies before Republican-led House panel 2. Claudia Sheinbaum, Mexico's first woman to be elected president 3. Hawaii volcano erupts, prompting officials to issue warning 4. Jury selection in Hunter Biden's federal gun trial begins 5. Biden executive order restricting asylum along border expected Tuesday")],
 'output': "The top stories on CBS News include: 1. Dr. Anthony Fauci testifies before Republican-led H